# II. Data Cleaning

### In this notebook I merge dataframes, clean, and do some  feature selection.

In [1]:
import pandas as pd
import pickle
from mvp_functions.functions import *
import numpy as np

#Import .pkl files
player_stats = pd.read_pickle('PlayerCareerStats.pkl')
team_stats = pd.read_pickle('TeamYearByYearStats.pkl')
player_awards = pd.read_pickle('PlayerAwards.pkl')

In [2]:
#Merge player and team stats
team_stats.rename(columns={'YEAR':'SEASON_ID'}, inplace=True)
players_teams = player_stats.merge(team_stats, how='inner', on=['TEAM_ID', 'SEASON_ID'],suffixes=('_player','_team'))

#Custom function that takes a df and adds player of the week and player of the month columns from awards_df
award_freq(players_teams, player_awards)

100%|██████████████████████████████████| 13718/13718 [00:00<00:00, 18432.85it/s]


In [3]:
#Merge mvp column from player_awards with player_teams to create final df
mvp = player_awards[player_awards['DESCRIPTION']=='NBA Most Valuable Player'].rename(columns={'PERSON_ID':'PLAYER_ID','SEASON':'SEASON_ID','TYPE':'MVP'})
df = players_teams.merge(mvp.loc[:,['PLAYER_ID','SEASON_ID','MVP']],how='left',on=['PLAYER_ID', 'SEASON_ID'])
df['MVP'] = df.MVP.map({'Award':1,np.NaN:0})

#Drop redundant columns and convert season column to single digit
df.drop(columns=['GS', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION','TEAM_CITY', 'TEAM_NAME', 'NBA_FINALS_APPEARANCE', 'CONF_COUNT', 'DIV_COUNT', 'PO_WINS'],inplace=True)
df['SEASON_ID'] = df['SEASON_ID'].map(lambda x: int(x.split('-',1)[0]))

#Convert object columns to float
convert_float(df)

pickle.dump(df, open('df.pkl','wb'))